In [1]:
import pandas as pd
from methods import get_result_data

%matplotlib inline 

In [2]:
main_path = "data/"
dirs = ["CO", "NO2", "O3", "PM10", "PM2.5", "SO2"]

In [4]:
df = get_result_data(main_path, dirs)

CO - Количество ошибочных значений: 4642 в процентном отношении 3.31
NO2 - Количество ошибочных значений: 19470 в процентном отношении 2.28
O3 - Количество ошибочных значений: 19807 в процентном отношении 8.37
PM10 - Количество ошибочных значений: 8467 в процентном отношении 1.31
PM2.5 - Количество ошибочных значений: 2626 в процентном отношении 2.6
SO2 - Количество ошибочных значений: 71863 в процентном отношении 54.66


In [5]:
df.head()

,CO_Concentration,NO2_Concentration,O3_Concentration,PM10_Concentration,PM2.5_Concentration,SO2_Concentration,Date,SO2_index,PM10_index,PM2.5_index,O3_index,NO2_index,CO_index,aqi_index
0,0.511042,30.759775,49.709444,204.521183,42.600000,13.191861,2013-01-01,7.192945,125.510484,118.482412,46.027263,15.435455,0.005807,125
1,0.513333,26.054568,52.687887,15.570061,25.800000,1.925773,2013-01-02,1.050040,14.416723,79.811159,48.785081,13.074352,0.005833,79
2,0.473333,31.372220,40.397083,13.538415,20.046667,2.064878,2013-01-03,1.125888,12.535570,67.711874,37.404707,15.742784,0.005379,67
3,0.479583,27.555411,55.400000,15.922422,17.530000,3.294757,2013-01-04,1.796487,14.742984,62.419313,52.306667,13.827484,0.005450,62
4,0.320208,25.538422,54.221250,10.792538,15.092000,1.299010,2013-01-05,0.708294,9.993091,57.292189,48.456083,12.815346,0.003639,57
